In [10]:
import codecs
from codecs import open
import re

sqlfile = "Customer Churn - W-DE - FSD\\tempfsrus.sql"
sql = open(sqlfile, mode='r', encoding='utf-8-sig').read()
#curDest.execute(sql)

In [11]:
import os

In [12]:
path = "Customer_Churn_v2_churn"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

Files and directories in ' Customer_Churn_v2_churn ' :
['CustomerChurn.sql', 'DelayOrders.sql', 'DistributionList.sql', 'MSO0.sql', 'MSO1.sql', 'MSOAggregated.sql', 'MSOPlatform.sql', 'MSOProductFamily.sql', 'MSOSegment.sql', 'NPS.sql', 'OrderOrigin.sql', 'PaymentDescription.sql', 'PreInforLOE.sql', 'Pricelist.sql', 'Returns - BlankOrders.sql', 'Returns.sql', 'ReturnsAggregated.sql', 'ReturnsPlatform.sql', 'ReturnsPlatformAggregated.sql', 'ReturnsProductFamily.sql', 'ReturnsProductFamilyAggregated.sql', 'ReturnsSegment.sql', 'ReturnsSegmentAggregated.sql', 'Sales-BlankOrders.sql', 'Sales.sql', 'SalesAggregated.sql', 'SalesCallAndVisitinlastweek.sql', 'SalesDistribution.sql', 'SalesforceStats.sql', 'SalesPlatform.sql', 'SalesPlatformAggregated.sql', 'SalesPriceList.sql', 'SalesProductFamily.sql', 'SalesProductFamilyAggregated.sql', 'SalesSegment.sql', 'SalesSegmentAggregated.sql', 'SalesVisitandCalls.sql']


In [13]:
for i in dir_list:
    sqlfile = "Customer_Churn_v2_churn\\"+i
    sql = open(sqlfile, mode='r', encoding='utf-8-sig').read()
    print(sql)
    # 1.Find the spaces in b/w square bracket and replace space with '_',  remove square bracket
    res = re.findall(r'\[.*?\]', sql)
    for j in set(res):
        # type(i)
        #print(i,i.replace(' ','_').replace(']','').replace('[',''))
        sql=sql.replace(str(j),str(j.replace(' ','_').replace(']','').replace('[','')))
    #3.remove sales.,master.,crm.,(nolock)  declare list of removeble items 
    remove_list=['sales.','master.','crm.','(nolock)']
    for k in remove_list:
        #print(i)
        sql=sql.replace(k,'')
    with open(i+'_updated.sql', 'w') as file:
        file.write(sql)

SET NOCOUNT ON;
SET ANSI_WARNINGS OFF;
--Declare @Datefilter Date= cast(Dateadd(	Day,-4, getdate()) as date)
Declare @Datefilter Date= DATEADD(D,-(DATEPART(W,getdate())+1)%7,getdate())

--Query for distinct purcahse dates
	select distinct  a.[Customer Code1], a.ShiptoCustomerAffilateCode, a.OrderDate2
	into #temp1
		from tempFSRUS a
	where left (a.[Customer Code1],1) = 'C'
	and a.[Local Order Type Code] in ('50A','50B')
	and cast([Order Date] as date) <= @Datefilter

--Calcualting days diff between purchases 
select   a.[Customer Code1] as [Customer Code], a.ShiptoCustomerAffilateCode as [Affiliate Code], a.OrderDate2
,lag	(A.OrderDate2, 1) over ( partition by A.[Customer Code1], A.ShiptoCustomerAffilateCode order by A.orderdate2 desc) l1
, datediff(dd , orderdate2 , lag	(A.OrderDate2, 1) over ( partition by A.[Customer Code1], A.ShiptoCustomerAffilateCode order by A.orderdate2 desc) ) as daysdiff
,DATEDIFF(dd, OrderDate2, @Datefilter) daysfromtoday
into #temp2
from #temp1 a


select 


SET NOCOUNT ON;

select 
 a.[Customer Code1]  as [Customer Code]
, a.[Affiliate Code]
, Year(a.OrderDate1) Year
, MONTH(a.OrderDate1) Month
,a.[Order Type]
,a.[Segment]
, sum(a.IsDelay) NoOfDelays
, sum( a.IsEarly) NoOfEarly
, min( a.daysDelay) as MinDelayDays
, max( a.daysDelay) as MaxDelayDays
, cast( SUM(a.daysDelay) as float)/ (Count( Distinct CONCAT (a.[Maintenance Sales Order], [Maintenance Sales Order Line]))) as AvgDelay
, sum(a.Quantity) Quantity
, Count(Distinct a.[Maintenance Sales Order]) NoOfOrders
, Count( Distinct CONCAT (a.[Maintenance Sales Order], [Maintenance Sales Order Line])) NoOfOrderLines
, case when (Count( CONCAT (a.[Maintenance Sales Order], [Maintenance Sales Order Line])) > ( Count( Distinct CONCAT (a.[Maintenance Sales Order], [Maintenance Sales Order Line])))) then 1 else 0 end multipartorder
, count( distinct [Product Code]) NoOfProducts

from #tempMSO a
group by 
 a.[Customer Code1]
, a.[Affiliate Code]
, Year(a.OrderDate1)
, MONTH(a.OrderDate1)
,a.[Ord

--Declare @Datefilter Date= cast(Dateadd(	Day,-4, getdate()) as date)
Declare @Datefilter Date= DATEADD(D,-(DATEPART(W,getdate())+1)%7,getdate())
--Declare @Datefilter Date='2022-04-25'


Select 
 a.[Customer Code1]  as [Customer Code]
, a.ShiptoCustomerAffilateCode as [Affiliate Code]
,'Return&Rebill' as [Order Type]
,P.[product Family]
, sum(a.IsDelay) NoOfDelays
, sum( a.IsEarly) NoOfEarly
, min( a.daysDelay) as MinDelayDays
, max( a.daysDelay) as MaxDelayDays
, case when SUM(a.daysDelay) <> 0 then round(cast( SUM(a.daysDelay) as float)/ (Count( Distinct CONCAT (a.[order], [Order Line]))),3) else 0 end AvgDelay
, sum(a.Quantity) Quantity
,  round(sum(a.[Gross Amount Base]), 2) GrossAmountBase
,  round(sum(a.[Discount Amount Base]), 2) DiscountAmountBase
,  round(sum(a.[Net Amount Base]), 2) NetAmountBase
,  round(sum(a.[Cost Amount Base]), 2) CostAmountBase
, case when sum(Quantity)<>0 then round((sum(totalASP) / Sum(Quantity) ),2) else 0 end  ASPAmountBase
,  round(sum(a.[Margin Am


--Declare @Datefilter Date= cast(Dateadd(	Day,-4, getdate()) as date)
Declare @Datefilter Date= DATEADD(D,-(DATEPART(W,getdate())+1)%7,getdate())
--Declare @Datefilter Date='2022-04-25'


Select distinct 
 a.[Customer Code1]  as [Customer Code]
, a.ShiptoCustomerAffilateCode as [Affiliate Code]
,a.[Order Type]
,p.[Segment]
, sum(a.IsDelay) NoOfDelays
, sum( a.IsEarly) NoOfEarly
, min( a.daysDelay) as MinDelayDays
, max( a.daysDelay) as MaxDelayDays
, case when SUM(a.daysDelay)<>0 then cast( SUM(a.daysDelay) as float)/ (Count( Distinct CONCAT (a.[order], [Order Line]))) else 0 end AvgDelay
, sum(a.Quantity) Quantity
,  round(sum(a.[Gross Amount Base]), 2) GrossAmountBase
,  round(sum(a.[Discount Amount Base]), 2) DiscountAmountBase
,  round(sum(a.[Net Amount Base]), 2) NetAmountBase
,  round(sum(a.[Cost Amount Base]), 2) CostAmountBase
, round((sum(totalASP) / Sum(Quantity) ),2) as ASPAmountBase
,  round(sum(a.[Margin Amount Base]), 2) MarginAmountBase
, Count(Distinct a.[Order]) NoOfO

In [7]:
print(sql)

SET NOCOUNT ON;
SET ANSI_WARNINGS OFF;
--Declare @Datefilter Date= cast(Dateadd(	Day,-4, getdate()) as date)
Declare @Datefilter Date= DATEADD(D,-(DATEPART(W,getdate())+1)%7,getdate())	
	--Drop table #tempFSDDE1
	--Drop table #tempFSDDE2
	--Drop table #tempFSDDE
	--drop table #tempFSD1
	--drop table #tempFSD2
	--drop table  #tempperiodicityFSD
	--drop table #tempcustomerFSD
	--drop table #tempcustomerFSD1

--select
--FSD.*
----CASe when cast(Order_Date as date) = '1900-01-01' then cast(Invoice_Date as date) else cast(Order_Date as date) end OrderDate1
--, p.Brand
----, PL.Segment
----, PL.SubSegment
--, c.Region
--,CASe when cast(Order_Date as date) in ( '1900-01-01', '1990-01-01' ) then cast(Order_Date as date) else cast(Order_Date as date) end OrderDate2
--, case when cast(Planned_Receipt_Date as date) < CAST(Delivery_Date as date) then 1 else 0 end IsDelay
--, case when cast(Planned_Receipt_Date as date) > CAST(Delivery_Date as date) then 1 else 0 end IsEarly
--, case when cast(Plan

In [8]:
# print(re.sub("(\s)+(?=[^[]*?\]\])","_",sql))
# sql.replace(/\[.*?\]/g, function(string) {
#     return string.replace(/\s/g, '');
# })
# m = re.sub(r'\s(?=[^\[\]]*]])', "_", s)
# print(m)
# import re
# #>>> text = 'i want to go to the[  super mall  ](place)'
# print(re.sub(r'(\w)\[\s+(.*?)\s+\]', r'\1 [\2]', sql))
# print(re.sub("(\s)+(?=[^[]*?\]\])","_",sql))
# print(sql.replace("Order Date","Order_Date"))
# print(sql.replace("Order Date","Order_Date"))
# print(sql.replace("Order Date","Order_Date"))
# m=re.search(r"\[([A-Za-z0-9_]+)\]", sql)
# m.group(1)

In [5]:
# 1.Find the spaces in b/w square bracket and replace space with '_',  remove square bracket
res = re.findall(r'\[.*?\]', sql)
for i in set(res):
#     type(i)
#     print(i,i.replace(' ','_').replace(']','').replace('[',''))
    sql=sql.replace(str(i),str(i.replace(' ','_').replace(']','').replace('[','')))

In [6]:
print(sql)


select  FSR.*,
--CASe when cast(Order_Date as date) = '1900-01-01' then cast(Invoice_Date as date) else cast(Order_Date as date) end OrderDate1
Invoice_Date as OrderDate1
, p.Brand
, PL.Segment
, PL.SubSegment
, c.Region
,CASe when cast(Order_Date as date) in ( '1900-01-01', '1990-01-01') then cast(Transaction_Date as date) else cast(Order_Date as date) end OrderDate2
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then 1 else 0 end IsDelay
, case when cast(Planned_Receipt_Date as date) > CAST(Actual_Delivery_Date as date) then 1 else 0 end IsEarly
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then DATEDIFF(dd,cast(Planned_Receipt_Date as date), CAST(Actual_Delivery_Date as date)) else 0 end DaysDelay
, OT.Order_Type
, OT.Local_Order_Type_Code
, OS.Order_Source
, C.Customer_Code as Customer_Code1
, fsr.Customer_Code fsrcustomercode
,C.Affiliate_Code ShiptoCustomerAffilateCode

into #tempFSRUS
from sales.factSale

In [7]:
#3.remove sales.,master.,crm.,(nolock)  declare list of removeble items 
remove_list=['sales.','master.','crm.','(nolock)']
for i in remove_list:
    print(i)
    sql=sql.replace(i,'')

sales.
master.
crm.
(nolock)


In [8]:
print(sql)


select  FSR.*,
--CASe when cast(Order_Date as date) = '1900-01-01' then cast(Invoice_Date as date) else cast(Order_Date as date) end OrderDate1
Invoice_Date as OrderDate1
, p.Brand
, PL.Segment
, PL.SubSegment
, c.Region
,CASe when cast(Order_Date as date) in ( '1900-01-01', '1990-01-01') then cast(Transaction_Date as date) else cast(Order_Date as date) end OrderDate2
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then 1 else 0 end IsDelay
, case when cast(Planned_Receipt_Date as date) > CAST(Actual_Delivery_Date as date) then 1 else 0 end IsEarly
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then DATEDIFF(dd,cast(Planned_Receipt_Date as date), CAST(Actual_Delivery_Date as date)) else 0 end DaysDelay
, OT.Order_Type
, OT.Local_Order_Type_Code
, OS.Order_Source
, C.Customer_Code as Customer_Code1
, fsr.Customer_Code fsrcustomercode
,C.Affiliate_Code ShiptoCustomerAffilateCode

into #tempFSRUS
from factSalesReven

In [9]:
# #sql=sql.replace(u"\u00A0", " ")
# sql = sql.replace("\xc2\xa0", " ")


In [10]:
# cleaned_string = sql.encode('ascii', 'ignore')


In [11]:
# sub1 = "join"
# len_sub = len(sub1)
# sub2 = "on"

In [12]:
# sql[0:].find(sub1)!=-1 and 0!=-1

In [13]:
# Lowercase string b/w join and ends with on 
starting_index = 0
l1_idx1 =0
l1_idx2 =0
lst = []
sub1 = "join"
len_sub = len(sub1)
sub2 = " on"
cnt = 1
while sql[starting_index:].find(sub1)!=-1 and l1_idx2!=-1:
    
    print('starting_index : ',starting_index)
    
    l1_idx1 = starting_index + sql[starting_index:].find(sub1)
    l1_idx2 = sql[l1_idx1:].find(sub2)
    print('l1_idx1 : ',l1_idx1)
    print('l1_idx2 : ',l1_idx1 + l1_idx2)
    print(l1_idx2)

    print("The extracted string : " + sql[l1_idx1 + len_sub:l1_idx1 + l1_idx2])
    if l1_idx1!=-1 and l1_idx2!=-1:
          lst.append(sql[l1_idx1 + len_sub:l1_idx1 + l1_idx2])



    print('-----------------------------------------------')



    starting_index = l1_idx1 + l1_idx2 + 1
    cnt=cnt+1



print(lst)

starting_index :  0
l1_idx1 :  1034
l1_idx2 :  1052
18
The extracted string :  dimproducts p
-----------------------------------------------
starting_index :  1053
l1_idx1 :  1090
l1_idx2 :  1121
31
The extracted string :  dimDistributionChannels  d
-----------------------------------------------
starting_index :  1122
l1_idx1 :  1183
l1_idx2 :  1206
23
The extracted string :  dimPriceLists   PL
-----------------------------------------------
starting_index :  1207
l1_idx1 :  1249
l1_idx2 :  1270
21
The extracted string :  dimCustomers C  
-----------------------------------------------
starting_index :  1271
l1_idx1 :  1316
l1_idx2 :  1338
22
The extracted string :  dimOrderTypes OT 
-----------------------------------------------
starting_index :  1339
l1_idx1 :  1381
l1_idx2 :  1405
24
The extracted string :  dimOrderSources  OS
-----------------------------------------------
[' dimproducts p', ' dimDistributionChannels  d', ' dimPriceLists   PL', ' dimCustomers C  ', ' dimOrderType

In [14]:
lst

[' dimproducts p',
 ' dimDistributionChannels  d',
 ' dimPriceLists   PL',
 ' dimCustomers C  ',
 ' dimOrderTypes OT ',
 ' dimOrderSources  OS']

In [15]:
for i in lst:
    if 'select' not in i:
        if 'Select' not in i:
            sql=sql.replace(i,i.lower())
            #print(i,i.lower())

In [16]:
print(sql)


select  FSR.*,
--CASe when cast(Order_Date as date) = '1900-01-01' then cast(Invoice_Date as date) else cast(Order_Date as date) end OrderDate1
Invoice_Date as OrderDate1
, p.Brand
, PL.Segment
, PL.SubSegment
, c.Region
,CASe when cast(Order_Date as date) in ( '1900-01-01', '1990-01-01') then cast(Transaction_Date as date) else cast(Order_Date as date) end OrderDate2
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then 1 else 0 end IsDelay
, case when cast(Planned_Receipt_Date as date) > CAST(Actual_Delivery_Date as date) then 1 else 0 end IsEarly
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then DATEDIFF(dd,cast(Planned_Receipt_Date as date), CAST(Actual_Delivery_Date as date)) else 0 end DaysDelay
, OT.Order_Type
, OT.Local_Order_Type_Code
, OS.Order_Source
, C.Customer_Code as Customer_Code1
, fsr.Customer_Code fsrcustomercode
,C.Affiliate_Code ShiptoCustomerAffilateCode

into #tempFSRUS
from factSalesReven

In [17]:
#replace "dd" in DATEDIFF(dd, date 1, date2 ) with blank
sql=sql.replace('dd,','')
#replace "dd" in DATEDIFF(dd, date 1, date2 ) with blank
sql=sql.replace('DAY,','')
#replace getdate() with current_date()
sql=sql.replace('getdate()','current_date()')
#replave STDEV with stddev
sql=sql.replace('STDEV','stddev')
#replace isnull with coalesce
sql=sql.replace('isnull','coalesce')

In [18]:
print(sql)


select  FSR.*,
--CASe when cast(Order_Date as date) = '1900-01-01' then cast(Invoice_Date as date) else cast(Order_Date as date) end OrderDate1
Invoice_Date as OrderDate1
, p.Brand
, PL.Segment
, PL.SubSegment
, c.Region
,CASe when cast(Order_Date as date) in ( '1900-01-01', '1990-01-01') then cast(Transaction_Date as date) else cast(Order_Date as date) end OrderDate2
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then 1 else 0 end IsDelay
, case when cast(Planned_Receipt_Date as date) > CAST(Actual_Delivery_Date as date) then 1 else 0 end IsEarly
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then DATEDIFF(cast(Planned_Receipt_Date as date), CAST(Actual_Delivery_Date as date)) else 0 end DaysDelay
, OT.Order_Type
, OT.Local_Order_Type_Code
, OS.Order_Source
, C.Customer_Code as Customer_Code1
, fsr.Customer_Code fsrcustomercode
,C.Affiliate_Code ShiptoCustomerAffilateCode

into #tempFSRUS
from factSalesRevenue 

In [42]:
#1st=[start='from',end=['join','left','right','inner']], 

In [41]:
# Lowercase string b/w join and ends with on 
starting_index = 0
l1_idx1 =0
l1_idx2 =0
lst = []
sub1 = "from"
len_sub = len(sub1)
sub2 = "join"
cnt = 1
while sql[starting_index:].find(sub1)!=-1 and l1_idx2!=-1:
    
    print('starting_index : ',starting_index)
    
    l1_idx1 = starting_index + sql[starting_index:].find(sub1)
    l1_idx2 = sql[l1_idx1:].find(sub2)
    print('l1_idx1 : ',l1_idx1)
    print('l1_idx2 : ',l1_idx1 + l1_idx2)
    print(l1_idx2)

    print("The extracted string : " + sql[l1_idx1 + len_sub:l1_idx1 + l1_idx2])
    if l1_idx1!=-1 and l1_idx2!=-1:
          lst.append(sql[l1_idx1 + len_sub:l1_idx1 + l1_idx2])



    print('-----------------------------------------------')



    starting_index = l1_idx1 + l1_idx2 + 1
    cnt=cnt+1



print(lst)

starting_index :  0
l1_idx1 :  998
l1_idx2 :  1031
33
The extracted string :  factSalesRevenue  FSR
left 
-----------------------------------------------
[' factSalesRevenue  FSR\r\nleft ']


In [43]:
for i in lst:
    sql=sql.replace(i,i.lower())

In [45]:
print(sql)


select  FSR.*,
--CASe when cast(Order_Date as date) = '1900-01-01' then cast(Invoice_Date as date) else cast(Order_Date as date) end OrderDate1
Invoice_Date as OrderDate1
, p.Brand
, PL.Segment
, PL.SubSegment
, c.Region
,CASe when cast(Order_Date as date) in ( '1900-01-01', '1990-01-01') then cast(Transaction_Date as date) else cast(Order_Date as date) end OrderDate2
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then 1 else 0 end IsDelay
, case when cast(Planned_Receipt_Date as date) > CAST(Actual_Delivery_Date as date) then 1 else 0 end IsEarly
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then DATEDIFF(cast(Planned_Receipt_Date as date), CAST(Actual_Delivery_Date as date)) else 0 end DaysDelay
, OT.Order_Type
, OT.Local_Order_Type_Code
, OS.Order_Source
, C.Customer_Code as Customer_Code1
, fsr.Customer_Code fsrcustomercode
,C.Affiliate_Code ShiptoCustomerAffilateCode

into #tempFSRUS
from factsalesrevenue 

In [ ]:
#1  comma space  end with dot
#2comma end with .
#start with space on end with dot

#do with lower case
#

#take the data after where or group by or oredr by clauses 
#start from where ends with dot 
#start from and end with dot
#

In [34]:
lst[9]

'as Customer_Code1\r\n, fsr'

In [46]:
# Lowercase string b/w join and ends with on 
starting_index = 0
l1_idx1 =0
l1_idx2 =0
lst = []
sub1 = ", "
len_sub = len(sub1)
sub2 = "."
cnt = 1
while sql[starting_index:].find(sub1)!=-1 and l1_idx2!=-1:
    
    print('starting_index : ',starting_index)
    
    l1_idx1 = starting_index + sql[starting_index:].find(sub1)
    l1_idx2 = sql[l1_idx1:].find(sub2)
    print('l1_idx1 : ',l1_idx1)
    print('l1_idx2 : ',l1_idx1 + l1_idx2)
    print(l1_idx2)

    print("The extracted string : " + sql[l1_idx1 + len_sub:l1_idx1 + l1_idx2])
    if l1_idx1!=-1 and l1_idx2!=-1:
          lst.append(sql[l1_idx1 + len_sub:l1_idx1 + l1_idx2])



    print('-----------------------------------------------')



    starting_index = l1_idx1 + l1_idx2 + 1
    cnt=cnt+1



print(lst)

starting_index :  0
l1_idx1 :  176
l1_idx2 :  179
3
The extracted string : p
-----------------------------------------------
starting_index :  180
l1_idx1 :  187
l1_idx2 :  191
4
The extracted string : PL
-----------------------------------------------
starting_index :  192
l1_idx1 :  201
l1_idx2 :  205
4
The extracted string : PL
-----------------------------------------------
starting_index :  206
l1_idx1 :  218
l1_idx2 :  221
3
The extracted string : c
-----------------------------------------------
starting_index :  222
l1_idx1 :  283
l1_idx2 :  799
516
The extracted string : '1990-01-01') then cast(Transaction_Date as date) else cast(Order_Date as date) end OrderDate2
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then 1 else 0 end IsDelay
, case when cast(Planned_Receipt_Date as date) > CAST(Actual_Delivery_Date as date) then 1 else 0 end IsEarly
, case when cast(Planned_Receipt_Date as date) < CAST(Actual_Delivery_Date as date) then DATEDIFF(

In [ ]:
# Hadf a problem with replacing aliaces with some information will lost in between the wordso

In [53]:
for i in lst:
    if '(' not in i:
        sql=sql.replace(i,i.lower())


In [54]:
print(sql)


select  FSR.*,
--cASe when cast(Order_Date as date) = '1900-01-01' then cast(Invoice_Date as date) else cast(Order_Date as date) end OrderDate1
Invoice_Date as OrderDate1
, p.Brand
, pl.Segment
, pl.SubSegment
, c.Region
,cASe when cast(Order_Date as date) in ( '1900-01-01', '1990-01-01') then cast(Transaction_Date as date) else cast(Order_Date as date) end OrderDate2
, case when cast(Planned_Receipt_Date as date) < cAST(Actual_Delivery_Date as date) then 1 else 0 end IsDelay
, case when cast(Planned_Receipt_Date as date) > cAST(Actual_Delivery_Date as date) then 1 else 0 end IsEarly
, case when cast(Planned_Receipt_Date as date) < cAST(Actual_Delivery_Date as date) then DATEDIFF(cast(Planned_Receipt_Date as date), cAST(Actual_Delivery_Date as date)) else 0 end DaysDelay
, ot.Order_Type
, ot.Local_Order_Type_code
, os.Order_Source
, c.customer_code as customer_code1
, fsr.customer_code fsrcustomercode
,c.Affiliate_code ShiptocustomerAffilatecode

into #tempFSRUS
from factsalesrevenue 